In [1]:
import pandas as pd

# llegim dataset original

df = pd.read_csv("qualitat_aire.csv")

df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y')

df = df[df['DATA'].dt.year > 2010]

hours_columns = ['01h', '02h', '03h', '04h', '05h', '06h', '07h', '08h',
                 '09h', '10h', '11h', '12h', '13h', '14h', '15h', '16h',
                 '17h', '18h', '19h', '20h', '21h', '22h', '23h', '24h']

# Calcula la mitjana diària i el màxim diari

df['MITJANA_DIARIA'] = df[hours_columns].mean(axis=1, skipna=True)

df['MAXIM_DIARI'] = df[hours_columns].max(axis=1, skipna=True)

# Calcula el valor màxim de la mitjana de totes les combinacions de 8 hores

rolling_mean = df[hours_columns].rolling(window=8, axis=1, min_periods=1).mean()

df['MAXIM_8H'] = rolling_mean.max(axis=1, skipna=True)


# Eliminem els les files dels municipis que continguin poques dades

def menys_de_100_files(grup):
    return len(grup) < 100

df_filtrat = df.groupby(['MUNICIPI', 'CONTAMINANT', df['DATA'].dt.year]).filter(menys_de_100_files)

df = df.drop(df_filtrat.index)

/tmp/ipykernel_65036/2025953241.py:5: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("qualitat_aire.csv")


In [2]:
import numpy as np

df3 = df[df['TIPUS ESTACIO'] != 'Industrial']

columnes_seleccionades = ['DATA', 'MUNICIPI', 'CONTAMINANT', 'MITJANA_DIARIA', 'MAXIM_DIARI', 'MAXIM_8H', 'LATITUD', 'LONGITUD']

df3 = df3[columnes_seleccionades]

contaminants_seleccionats = ['NO2', 'O3', 'PM10', 'PM2.5']

df3 = df3[df3['CONTAMINANT'].isin(contaminants_seleccionats)]

df_mitjana_per_municipi_2 = df3.groupby(['DATA', 'MUNICIPI', 'CONTAMINANT']).agg({'MITJANA_DIARIA': 'mean', 'MAXIM_8H': 'mean', 'MAXIM_DIARI': 'max'}).reset_index()


df_mitjana_per_municipi_2['MAXIM_DIARI'] = df_mitjana_per_municipi_2['MAXIM_DIARI']+0.000001


df_mitjana_per_municipi_2.to_csv("municipi_mitjana.csv", index=False)

df_mitjana_per_municipi_2

,DATA,MUNICIPI,CONTAMINANT,MITJANA_DIARIA,MAXIM_8H,MAXIM_DIARI
0,2011-01-01,Agullana,O3,45.916667,57.500000,79.000001
1,2011-01-01,Alcover,NO2,11.130435,16.625000,21.000001
2,2011-01-01,Alcover,O3,38.208333,51.500000,61.000001
3,2011-01-01,"Ametlla de Mar, l'",NO2,2.583333,7.666667,8.000001
4,2011-01-01,"Ametlla de Mar, l'",O3,36.750000,46.000000,52.000001
...,...,...,...,...,...,...
577875,2023-11-30,Vilafranca del Penedès,PM10,17.500000,24.000000,24.000001
577876,2023-11-30,Vilanova i la Geltrú,NO2,23.250000,26.000000,26.000001
577877,2023-11-30,Vilanova i la Geltrú,O3,8.250000,12.000000,12.000001
577878,2023-11-30,Vilanova i la Geltrú,PM10,14.000000,17.000000,17.000001


In [4]:
descripcio_contaminants = {
    'Contaminant': ['PM10', 'PM2.5', 'NO2', 'O3'],
    'Descripció': [
        'Les partícules en suspensió (PM) consisteixen en una complexa mescla de partícules líquides i sòlides de substàncies orgàniques i inorgàniques suspeses en l\'aire. Aquestes partícules es classifiquen en funció del diàmetre aerodinàmic: les partícules en suspensió de menys de 10 micres (PM10) i les partícules en suspensió de menys de 2,5 micres (PM2,5). Totes aquestes partícules provenen de les emissions a l\'aire per part de diferents focus: trànsit, indústria, emissions domèstiques o fonts naturals (per exemple la pols africana).',
        'Les partícules en suspensió (PM) consisteixen en una complexa mescla de partícules líquides i sòlides de substàncies orgàniques i inorgàniques suspeses en l\'aire. Aquestes partícules es classifiquen en funció del diàmetre aerodinàmic: les partícules en suspensió de menys de 10 micres (PM10) i les partícules en suspensió de menys de 2,5 micres (PM2,5). Totes aquestes partícules provenen de les emissions a l\'aire per part de diferents focus: trànsit, indústria, emissions domèstiques o fonts naturals (per exemple la pols africana).',
        'Els òxids de nitrogen (NOx) són un grup de compostos químics gasosos molt reactius. Els més importants són l\'òxid nítric (NO) i el diòxid de nitrogen (NO2). La major part dels NOx s\'emeten a l\'atmosfera en forma de NO que s\'oxida ràpidament a NO2. D\'altra banda, el NO2 és la principal font dels aerosols de nitrats que constitueixen una part important de les PM2,5 i, a més, el NO2 en presència de la llum ultraviolada és la font principal de l\'ozó troposfèric. Provenen d’emissions a l\'aire per trànsit terrestre, aeri i marítim; indústria. La quantitat emesa depèn de les condicions de la combustió i de la quantitat de combustible cremat.',
        'L’ozó troposfèric (O3) és un gas incolor i invisible que es troba de manera natural a l\'estratosfera, la capa de l\'atmosfera més propera a la terra, i que ens protegeix de les radiacions de rajos ultraviolats, i a la troposfera (tram d\'atmosfera situada entre el sòl i 10 km d\'altura). És un contaminant que no s\'emet directament a l\'atmosfera, sinó que es forma a partir de reaccions químiques entre contaminants primaris com els òxids de nitrogen i els compostos orgànics volàtils. És un contaminant secundari, és a dir, no és emès per cap focus. D\'origen fotoquímic, és a dir, es forma per l\'acció de la llum solar i en presència d\'òxids de nitrogen i hidrocarburs.'
    ],
    'Com es calcula': [
        'Per calcular l\'ICQA de les particules PM10, s\'utilitza la mitjana diària de tots els valors de transit i de fons de cada municipi',
        'Per calcular l\'ICQA de les particules PM2.5, s\'utilitza la mitjana diària de tots els valors de transit i de fons de cada municipi',
        'Per calcular l\'ICQA del Diòxid de Nitrògen, s\'utilitza el màxim horari del conjunt de valors de transit i de fons de cada municipi',
        'Per calcular l\'ICQA de l\'Ozó troposfèric, s\'utilitza la mitjana diària de les 8 hores amb valors més alts del conjunt d\'estacions de transit i de fons de cada municipi'
    ]
}

df4 = pd.DataFrame(descripcio_contaminants)
df4.to_csv("descripcio.csv", index=False)

